In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from pprint import pprint
import psycopg2

In [2]:
load_dotenv()

API_KEY = os.getenv('API_KEY')

BASE_URL = 'http://api.weatherstack.com/current'

params = {
    'access_key' : API_KEY,
    'query' : 'New York'
}

In [3]:
response = requests.get(BASE_URL, params = params)
if response.status_code == 200:
    data = response.json()

pprint(data)

{'current': {'cloudcover': 100,
             'feelslike': 7,
             'humidity': 89,
             'is_day': 'no',
             'observation_time': '08:52 AM',
             'precip': 0,
             'pressure': 1018,
             'temperature': 8,
             'uv_index': 0,
             'visibility': 4,
             'weather_code': 143,
             'weather_descriptions': ['Mist'],
             'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'],
             'wind_degree': 252,
             'wind_dir': 'WSW',
             'wind_speed': 5},
 'location': {'country': 'United States of America',
              'lat': '40.714',
              'localtime': '2024-12-10 03:52',
              'localtime_epoch': 1733802720,
              'lon': '-74.006',
              'name': 'New York',
              'region': 'New York',
              'timezone_id': 'America/New_York',
              'utc_offset': '-5.0'},
 'request': {'language': 'en',
 

In [6]:
weather = {
    'location' : data['location']['name'],
    'temperature' : data['current']['temperature'],
    'precip' : data['current']['precip'],
    'humidity' : data['current']['humidity'],
    'wind_speed' : data['current']['wind_speed'],
    'wind_degree' : data['current']['wind_degree'],
    'localtime' : data['location']['localtime']
}

weather

{'location': 'New York',
 'temperature': 8,
 'precip': 0,
 'humidity': 89,
 'wind_speed': 5,
 'wind_degree': 252,
 'localtime': '2024-12-10 03:52'}

In [7]:
df = pd.DataFrame([weather])

df

,location,temperature,precip,humidity,wind_speed,wind_degree,localtime
0,New York,8,0,89,5,252,2024-12-10 03:52


In [8]:
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']
weather_data = []

for location in locations:
    params = {
        'access_key' : API_KEY,
        'query' : location
        }
    
    
    response = requests.get(BASE_URL, params = params)
    if response.status_code == 200:
        data = response.json()
        weather_data.append(data)

pprint(weather_data)
    

[{'current': {'cloudcover': 100,
              'feelslike': 7,
              'humidity': 89,
              'is_day': 'no',
              'observation_time': '08:52 AM',
              'precip': 0,
              'pressure': 1018,
              'temperature': 8,
              'uv_index': 0,
              'visibility': 4,
              'weather_code': 143,
              'weather_descriptions': ['Mist'],
              'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'],
              'wind_degree': 252,
              'wind_dir': 'WSW',
              'wind_speed': 5},
  'location': {'country': 'United States of America',
               'lat': '40.714',
               'localtime': '2024-12-10 03:52',
               'localtime_epoch': 1733802720,
               'lon': '-74.006',
               'name': 'New York',
               'region': 'New York',
               'timezone_id': 'America/New_York',
               'utc_offset': '-5.0'},
  'requ

In [9]:
transformed_weather_data = []

for data in weather_data:
    transformed_weather_data.append({
        'location' : data['location']['name'],
        'temperature' : data['current']['temperature'],
        'precip' : data['current']['precip'],
        'humidity' : data['current']['humidity'],
        'wind_speed' : data['current']['wind_speed'],
        'wind_degree' : data['current']['wind_degree'],
        'local_datetime' : data['location']['localtime']
        })

transformed_weather_data


[{'location': 'New York',
  'temperature': 8,
  'precip': 0,
  'humidity': 89,
  'wind_speed': 5,
  'wind_degree': 252,
  'local_datetime': '2024-12-10 03:52'},
 {'location': 'Los Angeles',
  'temperature': 12,
  'precip': 0,
  'humidity': 83,
  'wind_speed': 8,
  'wind_degree': 17,
  'local_datetime': '2024-12-10 00:49'},
 {'location': 'Chicago',
  'temperature': 4,
  'precip': 0,
  'humidity': 57,
  'wind_speed': 17,
  'wind_degree': 286,
  'local_datetime': '2024-12-10 02:49'},
 {'location': 'Houston',
  'temperature': 18,
  'precip': 0,
  'humidity': 96,
  'wind_speed': 9,
  'wind_degree': 249,
  'local_datetime': '2024-12-10 02:29'},
 {'location': 'Phoenix',
  'temperature': 14,
  'precip': 0,
  'humidity': 7,
  'wind_speed': 14,
  'wind_degree': 15,
  'local_datetime': '2024-12-10 01:50'}]

In [10]:
weather_df = pd.DataFrame(transformed_weather_data)

weather_df

,location,temperature,precip,humidity,wind_speed,wind_degree,local_datetime
0,New York,8,0,89,5,252,2024-12-10 03:52
1,Los Angeles,12,0,83,8,17,2024-12-10 00:49
2,Chicago,4,0,57,17,286,2024-12-10 02:49
3,Houston,18,0,96,9,249,2024-12-10 02:29
4,Phoenix,14,0,7,14,15,2024-12-10 01:50


In [11]:
weather_df.columns

Index(['location', 'temperature', 'precip', 'humidity', 'wind_speed',
       'wind_degree', 'local_datetime'],
      dtype='object')

In [18]:
# Assign a unique primary key for tax_id using a dedicated numbering system
weather_df['weather_id'] = range(7, len(weather_df) + 7) # Sequential numbering starting at 7

weather_df = weather_df[['weather_id', 'location', 'temperature', 'precip', 'humidity', 'wind_speed', 'wind_degree', 'local_datetime']]

weather_df.head()

,weather_id,location,temperature,precip,humidity,wind_speed,wind_degree,local_datetime
0,7,New York,8,0,89,5,252,2024-12-10 03:52
1,8,Los Angeles,12,0,83,8,17,2024-12-10 00:49
2,9,Chicago,4,0,57,17,286,2024-12-10 02:49
3,10,Houston,18,0,96,9,249,2024-12-10 02:29
4,11,Phoenix,14,0,7,14,15,2024-12-10 01:50


In [19]:
# Load environment variables from the .env file
load_dotenv()


# Develop a function to get the Database connections
def get_db_connection():
    connection = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        host = os.getenv('DB_HOST')
    )

    return connection

# connect to the Database
conn = get_db_connection()

In [20]:
# Create a function that sets up the schema and tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
                            CREATE SCHEMA IF NOT EXISTS Weather_db;

                            DROP TABLE IF EXISTS Weather_db.Weather_Table CASCADE;
                            
                            CREATE TABLE Weather_db.Weather_Table (
                                weather_id INTEGER PRIMARY KEY,
                                location VARCHAR(100000),
                                temperature INTEGER,
                                precip INTEGER,
                                humidity INTEGER,
                                wind_speed INTEGER,
                                wind_degree INTEGER,
                                local_datetime  TIMESTAMP
                            );
                            '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

create_tables()

In [ ]:
### Loading the Data into the db Tables
conn = get_db_connection()
cursor = conn.cursor()

# Insert the dataframe into SQL tables
for _,row in weather_df.iterrows():
    cursor.execute(
        '''INSERT INTO Weather_db.Weather_Table (weather_id, location, temperature, precip, humidity, wind_speed, wind_degree, local_datetime)
        
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)''',
            (row['weather_id'], row['location'], row['temperature'], row['precip'], row['humidity'], row['wind_speed'], row['wind_degree'], row['local_datetime'])
        )
    

# Commit Changes
conn.commit()

# Close Connection
cursor.close()
conn.close()
    

In [26]:
display(weather_df.info())
display(weather_df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   weather_id      5 non-null      int64 
 1   location        5 non-null      object
 2   temperature     5 non-null      int64 
 3   precip          5 non-null      int64 
 4   humidity        5 non-null      int64 
 5   wind_speed      5 non-null      int64 
 6   wind_degree     5 non-null      int64 
 7   local_datetime  5 non-null      object
dtypes: int64(6), object(2)
memory usage: 452.0+ bytes


None

Index(['weather_id', 'location', 'temperature', 'precip', 'humidity',
       'wind_speed', 'wind_degree', 'local_datetime'],
      dtype='object')

In [27]:
weather_df.columns

Index(['weather_id', 'location', 'temperature', 'precip', 'humidity',
       'wind_speed', 'wind_degree', 'local_datetime'],
      dtype='object')

In [28]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   weather_id      5 non-null      int64 
 1   location        5 non-null      object
 2   temperature     5 non-null      int64 
 3   precip          5 non-null      int64 
 4   humidity        5 non-null      int64 
 5   wind_speed      5 non-null      int64 
 6   wind_degree     5 non-null      int64 
 7   local_datetime  5 non-null      object
dtypes: int64(6), object(2)
memory usage: 452.0+ bytes
